# Domain-specific Q&A with Hugging Face and LoRA
This notebook demonstrates a simple pipeline for fine-tuning a transformer-based model (e.g., LLaMA/GPT) on a domain-specific Q&A dataset using LoRA.

In [ ]:
# Install required libraries
!pip install -q transformers datasets peft accelerate bitsandbytes gradio

## Step 1: Load a Pretrained Model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "tiiuae/falcon-7b-instruct"  # Or any other model that supports LoRA
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

## Step 2: Apply LoRA with PEFT

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],  # Adjust based on model architecture
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

## Step 3: Load or Create a Simple Q&A Dataset

In [ ]:
from datasets import Dataset

# Example: A few Q&A pairs for training
data = {
    "question": [
        "What documents are needed for an insurance claim?",
        "How long is the cooling-off period?",
    ],
    "answer": [
        "Typically, a completed claim form, policy number, and supporting invoices or reports.",
        "Usually 14 days from the date of purchase or agreement."
    ]
}

dataset = Dataset.from_dict(data)
dataset = dataset.map(lambda x: {"text": f"Q: {x['question']} A: {x['answer']}"})
dataset

## Step 4: Train or Fine-tune the Model (Optional Demo Step)

In [ ]:
# Skipping full training for demo – insert Trainer setup here if needed.

## Step 5: Run Inference

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
pipe("Q: What documents are needed for an insurance claim? A:")

## Step 6: Deploy with Gradio

In [ ]:
import gradio as gr

def qa_bot(prompt):
    result = pipe(f"Q: {prompt} A:")[0]["generated_text"]
    return result.split("A:")[-1].strip()

gr.Interface(fn=qa_bot, inputs="text", outputs="text", title="Domain Q&A Bot").launch()